# Import necessary libraries

In [65]:
import pandas as pd
from pandas import DataFrame, Series
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from cStringIO import StringIO
from bs4 import BeautifulSoup
import urllib2
import urllib
import ast
from selenium import webdriver  
from selenium.common.exceptions import NoSuchElementException  
from selenium.webdriver.common.keys import Keys  
import time
import re
import json
import multiprocessing as mp
from HTMLParser import HTMLParser
import datetime
h = HTMLParser()

# Define basic functions used throughout

In [2]:
def isFloat(string):
    try:
        float(string)
        return True
    except ValueError:
        return False

In [3]:
def flatten(foo):
    for x in foo:
        if hasattr(x, '__iter__'):
            for y in flatten(x):
                yield y
        else:
            yield x

# Create function to navigate through the built-in javascript calendar

In [4]:
def navigate_calendar(date, browser):
        
    yearidx = date.year%10 + 2
    monthidx = date.month
        
    # Click on current month to open up menu of all months
    browser.find_element_by_xpath('/html/body/div[5]/div[1]/table/thead/tr[1]/th[3]').click()
    time.sleep(.5)
    # Click current year
    browser.find_element_by_xpath('/html/body/div[5]/div[2]/table/thead/tr/th[2]').click()
    time.sleep(.5)
    
    if date < datetime.date(2010,1,1):
        # Click for previous decade (2000-2009)
        browser.find_element_by_xpath('/html/body/div[5]/div[3]/table/thead/tr/th[1]').click()
        time.sleep(.5)
    if date < datetime.date(2000,1,1):
        # Click for previous decade (1990-1999)
        browser.find_element_by_xpath('/html/body/div[5]/div[3]/table/thead/tr/th[1]').click()
        time.sleep(.5)
    # Click desired year within decade (span[2] gets 1990)
    browser.find_element_by_xpath('/html/body/div[5]/div[3]/table/tbody/tr/td/span[%d]' %yearidx).click()
    time.sleep(.5)
    # Click month (span[3] gets March)
    browser.find_element_by_xpath('/html/body/div[5]/div[2]/table/tbody/tr/td/span[%d]' %monthidx).click()
    time.sleep(.5)

# Pull auction IDs, names, dates, and department and save into dataframe

This will be used to pull all individual auctions, with the ability to sort by department, date, and the specific auction it was sold in.

Auction ID is crucial in the URL to identify which event the items were sold in.

### NOTE: As Sotheby's uses JS for their auctions, we will have to use selenium to create a mock browser in order to pull this information.

In [5]:
def pull_auction_ids_names_dates_viaDept(departmentNum, endDate, run):

    browser = webdriver.Firefox() 
    browser.maximize_window()
    browser.get('http://www.sothebys.com/en/auctions/results.html')  
    time.sleep(3)
    # Click on advanced search 
    browser.find_element_by_xpath('//*[@id="bodyWrap"]/div[2]/section/form/div[1]/div/a').click() 
    time.sleep(2)
    
    # Fix start date - 1/22/2000 is the first date where they start uploading auction data
    startDate = datetime.date(2000,1,22)
    browser.find_element_by_xpath('//*[@id="bodyWrap"]/div[2]/section/form/div[2]/div/label[1]/div/input').click()
    time.sleep(2)
    navigate_calendar(startDate, browser)
    time.sleep(2)
    # Select 1/22/2000 in calendar view
    browser.find_element_by_xpath('/html/body/div[5]/div[1]/table/tbody/tr[4]/td[6]').click()
    time.sleep(2)
    
    # Check to see if endDate needs to be updated and fix if so, create dataframe if first run
    if endDate != datetime.date.today():
        browser.find_element_by_xpath('//*[@id="bodyWrap"]/div[2]/section/form/div[2]/div/label[2]/div/input').click()
        time.sleep(2)
        browser.find_element_by_xpath('//*[@id="bodyWrap"]/div[2]/section/form/div[2]/div/label[2]/div/input').click()
        time.sleep(2)
        navigate_calendar(endDate, browser)
        time.sleep(2)
        browser.find_element_by_xpath('/html/body/div[5]/div[1]/table/tbody/tr[6]/td[7]').click()
        time.sleep(2)
    else:
        browser.find_element_by_xpath('//*[@id="bodyWrap"]/div[2]/section/form/div[2]/div/label[2]/div/input').click()
        time.sleep(2)
        browser.find_element_by_xpath('//*[@id="bodyWrap"]/div[2]/section/form/div[2]/div/label[2]/div/input').click()
        time.sleep(2)
        browser.find_element_by_xpath('/html/body/div[5]/div[1]/table/tbody/tr[2]/td[7]').click()
        time.sleep(2)
        allAuctions = pd.DataFrame()
    
    # Apply search parameters
    browser.find_element_by_xpath('//*[@id="apply-btn"]').click()
    time.sleep(5)
    
    # Open Department Dialog
    browser.find_element_by_xpath('//*[@id="bodyWrap"]/div[2]/section/form/div[1]/div/label[2]/div/button/span').click()
    time.sleep(2)
    
    # Select Department
    if departmentNum > 14:
        numScrolls = (departmentNum-5)//2
        scroll = 1
        for i in range(0,numScrolls):
            browser.find_element_by_xpath('//*[@id="bodyWrap"]/div[2]/section/form/div[1]/div/label[2]/div/ul/li[%d]/a/label' % scroll).send_keys(Keys.ARROW_DOWN)
            time.sleep(.25)
            scroll += 2
            
    if departmentNum > 60:
        browser.find_element_by_xpath('//*[@id="bodyWrap"]/div[2]/section/form/div[1]/div/label[2]/div/ul/li[59]/a/label').send_keys(Keys.ARROW_DOWN)
        time.sleep(.25)
        browser.find_element_by_xpath('//*[@id="bodyWrap"]/div[2]/section/form/div[1]/div/label[2]/div/ul/li[64]/a/label').send_keys(Keys.ARROW_DOWN)
    
    time.sleep(1)
                                      
    browser.find_element_by_xpath('//*[@id="bodyWrap"]/div[2]/section/form/div[1]/div/label[2]/div/ul/li[%d]/a/label' %departmentNum).click()
    time.sleep(3)
    
    # Press View All
    browser.find_element_by_xpath('//*[@id="AuctionList"]/div/div[1]/div[2]/a[3]/span').click()
    time.sleep(5)
    
    # Extract source and close web browser
    html_source = browser.page_source  

    # Load in data into BeautifulSoup
    soup = BeautifulSoup(html_source,'html.parser')  
    auctionList = soup.findAll('article',{'class':'AuctionListItem'})  
    
    auctions_df = pd.DataFrame()
    anchor1 = "/en/auctions/"
    anchor2 = "title"
    anchor3 = "><img alt"
    anchor4 = "datetime"
    anchor5 = "</time>"
    
    # Find all auction IDs, names, dates
    
    for auction in auctionList:
        x = str(auction)
        x1 = x[x.find(anchor4)+len(anchor4)+3:x.find(anchor5)+1]
        auction_Id = x[x.find(anchor1)+len(anchor1)+5:x.find(anchor2)-7]
        auction_Name = x[x.find(anchor2)+7:x.find(anchor3)-1]
        auction_Date = x1[x1.find(">")+1:x1.find("<")]
        df2 = pd.DataFrame([[auction_Name, auction_Id, auction_Date]])
        
        # Exclude the auctions with no data ("Page not found")
        
        auctions_df = auctions_df.append(df2)

        
    # Format auctions dataframe
    
    auctions_df.columns = ['auctionName','auctionId','auctionDate']
    
    # Add to all other auctions of department
    
    allAuctions = allAuctions.append(auctions_df)
    
    # Return all auctions / rerun if haven't captured full category
    
    browser.quit()
    
    if auctions_df.shape[0]<500:
        allAuctions = allAuctions.drop_duplicates()
        allAuctions = allAuctions[allAuctions['auctionId'].map(lambda x: x[:5]) != 'logue']
        allAuctions.index = range(0,len(allAuctions))
        return allAuctions
    else:
        auctions.index = range(0,auctions.shape[0])
        newEndDate = auctions['auctionDate'][500*run-1]
        run += 1
        auction_ids_names_dates_viaDept(departmentNum, newEndDate, run)

# Now that we have all of the auctions ever hosted by Sotheby's, let's pull the specific items presented at these auctions and the relevant information on them.

### This function will allow us to open the page of a specific auction

In [95]:
def read_in_auction_source(auctionId):

    for year in range(2017,2018):
        testurl = "http://www.sothebys.com/en/auctions/" +  str(year) + "/" + auctionId + ".html"
        print(testurl)
        print(type(testurl))
        r = urllib2.urlopen(testurl)

        soup = BeautifulSoup(r,'html.parser')
        soup_s = str(soup)
        
        testCase = "Page Not Found"
        
        if soup_s[soup_s.find("<title>")+7:soup_s.find("<title>")+7+len(testCase)] != testCase:
            break
            
    return soup

In [97]:
testurl = 'http://www.sothebys.com/en/auctions/2017/master-paintings-19th-century-european-n09600.html'
read_in_auction_source(testurl)

http://www.sothebys.com/en/auctions/2017/http://www.sothebys.com/en/auctions/2017/master-paintings-19th-century-european-n09600.html.html
<type 'str'>


HTTPError: HTTP Error 400: Bad Request

### This function will permit us to pull the information on a specific item, including currency it was sold in, price it was sold for, etc.

In [7]:
def format_auction_data(soup_object, auctionName, auctionId, auctionDate):
    
    allLots = str(soup_object.find_all("script", type="text/javascript"))
    
    allLots = allLots[allLots.find('ECAT.lot['):allLots.find('cleanArray')]
    
    # There's an issue with the default input of the JSON object 
    # being in the improper formatting of booleans, fix.
    
    allLots = allLots.replace('false',"'False'")
    allLots = allLots.replace('true',"'True'")
    
    allLots = allLots.split('};')[:-1]
    
    # We had to split by '};' rather than just ';' as some of 
    # the strings within the JSON object properties contained semicolons.
    # Now we want to make sure that we retain the proper formatting by replacing the '}'
    
    allLots = [item + "}" for item in allLots]
    
    # Now we want to convert these objects to dictionaries so
    # that they are easier to access.
    
    allLots_d = []
    lotNum = 1

    for idx,item in enumerate(allLots):
        dig = len(str(lotNum))
        newDict = ast.literal_eval(item[13+dig:])
        allLots_d.append(newDict)
        lotNum += 1
        
    # Now we have all dictionaries held in a list, however we only 
    # need a fraction of the keys.

    newKeys = ['currency', 'highEst', 'lowEst', 'salePrice', 
               'title','guaranteeLine','genre','id','isSold',
               'medium','type']

    allLots_dn = allLots_d

    for idx, item in enumerate(allLots_dn):
        allLots_dn[idx] = {key: allLots_dn[idx][key] for key in newKeys}
        
    # Finally we should have our data in a format that is accessible. 
    # All dictionaries are held in a list of "allLots_dn".
    
    
    #### Can merge two for loops here and drop dataframe columns you're
    #### not using instead of dropping keys from dictionaries
    
    df = pd.DataFrame()

    for item in allLots_dn:
        df1 = pd.DataFrame(item.values()).transpose()
        df1.columns = item.keys()
        df = df.append(df1)

    df.index = range(0, len(df))
    df['auctionName'] = auctionName
    df['auctionId'] = auctionId
    df['auctionDate'] = auctionDate
    
    return df

NameError: global name 'Request' is not defined

In [8]:
def convert_auction_data(auctionName, auctionId, auctionDate):
    soupObj = read_in_auction_source(auctionId)
    auction_f = format_auction_data(soupObj,auctionName,auctionId, auctionDate)
    return auction_f

In [28]:
def pull_items_from_auction(row, df=masterdf_AuctionEvents):

    print("Pulling auction " + str(row+1) + " of " + str(df.shape[0]))
    auctionId = df['auctionId'][row]
    auctionName = df['auctionName'][row]
    auctionDate = df['auctionDate'][row]
    print(auctionName, auctionId, auctionDate)
    auctionData = convert_auction_data(auctionName, auctionId, auctionDate)
    return auctionData

In [21]:
endDate = datetime.date(2017,4,19)
masterdf_AuctionEvents = pull_auction_ids_names_dates_viaDept(51, endDate, 1)

In [22]:
masterdf_AuctionItems = pd.DataFrame()
numAuctions = masterdf_AuctionEvents.shape[0]
auctionData = []
    

In [29]:
read_in_auction_source()

Pulling auction 2 of 126
('Master Paintings &amp; 19th Century European Art', 'master-paintings-19th-century-european-n09600', '27 JAN 2017')


HTTPError: HTTP Error 400: Bad Request

In [26]:
if __name__ == '__main__':    
    
    pool = mp.Pool(mp.cpu_count())
    results = pool.map_async(pull_items_from_auction, range(0,numAuctions)).get(9999)
    masterdf_AuctionItems = pd.concat(results)

Pulling auction 9 of 126
Pulling auction 1 of 126
Pulling auction 17 of 126
Pulling auction 25 of 126
Pulling auction 33 of 126
Pulling auction 41 of 126
Pulling auction 49 of 126
Pulling auction 57 of 126
Pulling auction 73 of 126
Pulling auction 65 of 126


MaybeEncodingError: Error sending result: 'HTTPError()'. Reason: 'TypeError("can't pickle cStringIO.StringO objects",)'

Pulling auction 81 of 126
Pulling auction 89 of 126
Pulling auction 97 of 126
Pulling auction 105 of 126
Pulling auction 113 of 126
Pulling auction 121 of 126


# We should now have a dataframe with all of the items ever auctioned by Sotheby's from the year 2000 to present.

### Let's now format this properly and add a couple helpful columns

In [11]:
def format_aux_cols(df):
    df['auctionYear'] = df['auctionDate'].apply(lambda x: int(x[-4:]))

    for column in ['highEst', 'lowEst', 'salePrice']:
        df[column] = df[column].apply(lambda x: int(x))

    df.index = range(0,len(df))
    df['lot_desc'] = ""

# We're missing a lot of information on the items themselves. Let's extract their descriptions to obtain further information.

In [ ]:
def extract_lot_description(lot, auctionId, auctionYear):

    testURL = 'http://www.sothebys.com/en/auctions/ecatalogue/' + str(auctionYear) + '/' + auctionId + '/' + 'lot.' + str(lot) + '.html'
    
    r = urllib2.urlopen(testURL)
    soup = BeautifulSoup(r, "lxml")

    soup_s = str(soup)

    unwanted_tags = ['<br>','<br/>',
                     '<em>','</em>',
                     '<span>','</span>',
                     '<p>','</p>',
                     '<h1>','</h1>',
                     '<h2>','</h2>',
                     '<h3>','</h3>',
                     '<h4>','</h4>',
                     '<h5>','</h5>',
                     '<strong>','</strong>']
    
    for item in unwanted_tags:
        soup_s = soup_s.replace(item,'\n')

    soup_s = ''.join(i for i in soup_s if ord(i)<128)

    for line in soup_s:
        line = line.strip()
        line = line.encode("utf-8", 'i')
    
    anchor1 = '<div class="lotdetail-description-text">'
    anchor2 = '</div>'
    anchor3 = 'lotdetail-section-sub-header">'
    anchor4 = '</div>'

    if soup_s.find(anchor1) != -1:
        soup_s1 = soup_s[soup_s.find(anchor1)+len(anchor1):]
        soup_s2 = soup_s1[:soup_s1.find(anchor2)]
    else:
        soup_s2 = ''
        
    if soup_s.find(anchor3) != -1:
        soup_s3 = soup_s[soup_s.find(anchor3)+len(anchor3):]
        soup_s4 = soup_s3[:soup_s3.find(anchor4)]
    else:
        soup_s4 = ''
    
    soup_s5 = soup_s2 + soup_s4
    
    soup_s5 = soup_s5.replace("<h3 class=\"lotdetail-section-sub-header\">",'')

    description = ' '.join(soup_s5.split())
    
    return description

In [ ]:
def pull_all_descriptions(startRow, endRow, dataFrame):
    if input('are you sure you want to start the batch? all existing descriptions will be lost') == "yes":
        l = []
        time1 = time.time()
        
        for row in range(startRow,endRow):
            print "Pulling desc for lot #" + str(row)
            l.append(extract_lot_description(dataFrame['id'][row], dataFrame['auctionId'][row], dataFrame['auctionYear'][row]))

        dataFrame['lot_desc'][startRow:endRow] = l
        time2 = time.time()
        print("time to complete: " + str((time2-time1)/60/60) + " hours")
        print("time per problem: " + str((time2-time1)/(endRow-startRow)))

In [ ]:
import sys

# df = masterdf_AuctionItems

def distribute_lot_desc_extr(df, startrow, endrow):

    def runSimulation(row):

        print "Pulling desc for lot #" + str(row)
        description = extract_lot_description(df['id'][row], df['auctionId'][row],df['auctionYear'][row])
        return description

    if __name__ == '__main__':

        numRows = startrow-endrow
        pool = mp.Pool(mp.cpu_count())
        time1 = time.time()
        results = pool.map_async(runSimulation, range(0,numRows), chunksize=10).get(99999)

        time2 = time.time()
        print("time to complete: " + str((time2-time1)/60/60) + " hours")
        print("time per problem: " + str((time2-time1)/numRows))
        df['lot_desc'][range(0,numRows)] = results
